# Описание проекта

- Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

- В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

- Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

# План исследования 
      Шаг 1
- Откройте файл с данными и изучите его. Путь к файлу: datasets/users_behavior.csv. Скачать датасет.

      Шаг 2
- Разделите исходные данные на обучающую, валидационную и тестовую выборки.

      Шаг 3
- Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

      Шаг 4
- Проверьте качество модели на тестовой выборке.

      Шаг 5
- Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.


# Рекомендация тарифов
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных
#### Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
!pip install progress


  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9614 sha256=0778f64aae672cc7288e0504378f056a0e098e529583a083ee6e8bfcc8bf6f9f
  Stored in directory: /home/jovyan/.cache/pip/wheels/4c/9b/0a/a78ff56725af3ef70792f9ed0f8dbbc4c0315edc62cbc4a6b8
Successfully built progress


In [2]:
# Импортируем все нужные библиотеки для дальнейшей работы
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import time
from tqdm import tqdm
from progress.bar import Bar

In [3]:
# Откроем таблицу двумя путями
server_path = '/datasets/users_behavior.csv'
local_path = '/Users/mmaximmaximovgmail.com/Desktop/DS/project_6/users_behavior.csv'

try:
    link = pd.read_csv(server_path)
except:
    link = pd.read_csv(local_path)

# 1 Откройте и изучите файл

In [4]:
link.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
link.shape

(3214, 5)

In [6]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [7]:
link.isnull().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [8]:
link.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Мини вывод
- Все данные уже обработанны и готовы к выполнению последующих работ.
- В целом можно заменить тип данных столбцов `calls`, `messages` на тип int, но посмотрим в дальнейшем что выйдет.

# 2 Разбейте данные на выборки

In [9]:
# Создаем выборку обучающую и валидационную, делим исходные данные в соотношении 3:1:1. 
feature = link.drop('is_ultra', axis=1)
target = link['is_ultra']

In [10]:
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=0.4, random_state=12345)


In [11]:
feature_test, feature_valid, target_test, target_valid = train_test_split(feature_test, target_test, test_size=0.5, random_state=12345)


In [12]:
display(feature_train.shape, target_train.shape)
display(feature_test.shape, target_test.shape)
display(feature_valid.shape, target_valid.shape)

(1928, 4)

(1928,)

(643, 4)

(643,)

(643, 4)

(643,)

# 3 Исследуйте модели

In [13]:
# Начнем с дерева.

best_model = None
best_result = 0
for depth in tqdm(range(1, 20)):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(feature_train, target_train)
    prediction = model.predict(feature_valid)
    result = accuracy_score(target_valid, prediction)
    print('Глубина дереа:', depth, 'Точность:', result)
    if result > best_result:
        best_model=depth
        best_result=result
        
print('Подходящая глубина :', best_model, 'Подходящая точность:', best_result)

  0%|          | 0/19 [00:00<?, ?it/s]

Глубина дереа: 1 Точность: 0.7356143079315708
Глубина дереа: 2 Точность: 0.7744945567651633
Глубина дереа: 3 Точность: 0.7791601866251944
Глубина дереа: 4 Точность: 0.7744945567651633
Глубина дереа: 5 Точность: 0.7838258164852255
Глубина дереа: 6 Точность: 0.776049766718507
Глубина дереа: 7 Точность: 0.7993779160186625
Глубина дереа: 8 Точность: 0.7931570762052877


100%|██████████| 19/19 [00:00<00:00, 115.00it/s]

Глубина дереа: 9 Точность: 0.7807153965785381
Глубина дереа: 10 Точность: 0.7884914463452566
Глубина дереа: 11 Точность: 0.7744945567651633
Глубина дереа: 12 Точность: 0.7807153965785381
Глубина дереа: 13 Точность: 0.7713841368584758
Глубина дереа: 14 Точность: 0.76049766718507
Глубина дереа: 15 Точность: 0.7713841368584758
Глубина дереа: 16 Точность: 0.749611197511664
Глубина дереа: 17 Точность: 0.7480559875583204
Глубина дереа: 18 Точность: 0.749611197511664
Глубина дереа: 19 Точность: 0.7433903576982893
Подходящая глубина : 7 Подходящая точность: 0.7993779160186625


In [14]:
# Случайный лес
best_model = None
best_result = 0
for est in tqdm(range(1, 10)):
    model = RandomForestClassifier(random_state=12345, n_estimators=est, min_samples_leaf=2)
    model.fit(feature_train, target_train)
    result = model.score(feature_valid, target_valid)
    if result > best_result:
        best_model = est
        best_result = result
        
print('Кол-во деревьев в лесу :', best_model, 'Подходящая точность:', best_result)

100%|██████████| 9/9 [00:00<00:00, 42.85it/s]

Кол-во деревьев в лесу : 8 Подходящая точность: 0.7838258164852255


In [27]:
import itertools
import numpy as np

best_params = None
best_result = 0

for params in tqdm(itertools.product(range(1, 16), range(1, 21), range(1, 5))):
    depth, est, min_s_l = params
    model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est, min_samples_leaf=min_s_l)
    model.fit(feature_train, target_train)
    result = model.score(feature_valid, target_valid)
    if result > best_result:
        best_params = params
        best_result = result
        
print(f'Score of best model {np.round(best_result, 5)}')
print(f'\nParams:\nmax_depth = {best_params[0]}\nn_estimators = {best_params[1]}\nmin_samples_leaf = {best_params[2]}')

1200it [00:39, 30.38it/s]

Score of best model 0.8196

Params:
max_depth = 11
n_estimators = 6
min_samples_leaf = 3


<div style="border:solid purple 5px; padding: 20px"> 
<h2 align="center"> Рубрика «Питонячий лайфхакер» <a class="tocSkip"> </h2>

<h3> Индикаторы состояния: progress и tqdm <a class="tocSkip"> </h3>

Как перестать беспокоиться о том, не завис ли скрипт? Вы можете создавать свои индикаторы состояния. Это весьма увлекательное занятие. Но делать то же самое с [progress](https://pypi.org/project/progress/) или [tqdm](https://pypi.org/project/tqdm/) куда быстрее и надежнее.

Обычно я использую эти две библиотеки, но вот [здесь](https://habr.com/ru/post/483400/) ты можешь почитать про альтернативные варианты

![](https://i.ibb.co/B4SDm8Y/68747470733a2f2f7261772e6769746875622e636f6d2f7665726967616b2f70726f67726573732f6d61737465722f64656d.gif)

In [16]:
# Логистическая реграссия
model = LogisticRegression(solver='lbfgs', penalty='l2')
model.fit(feature_train, target_train)
result = model.score(feature_valid, target_valid)
print('Accuracy:', result)

Accuracy: 0.6842923794712286


### Мини вывод по 3 шагу
- Лучший показатель показала модель случайного леса и дерево, логистическая регрессия не вписывается в наши 0.75 .

# 4 Проверьте модель на тестовой выборке

In [17]:
# Тест дерева
model = DecisionTreeClassifier(max_depth=7)
model.fit(feature_train, target_train)
test_predictions=model.predict(feature_test)
print('Тестовая выборка:', accuracy_score(target_test, test_predictions))

Тестовая выборка: 0.7822706065318819


In [18]:
# Тест случайного леса
model = RandomForestClassifier(random_state=12345, min_samples_leaf=2, n_estimators=8)
model.fit(feature_train, target_train)
test_predictions=model.predict(feature_test)
print('Тестовая выборка:', accuracy_score(target_test, test_predictions))


Тестовая выборка: 0.7744945567651633


In [19]:
# Тест случайной логистическогой регрессии
model = LogisticRegression(solver='lbfgs', penalty='l2')
model.fit(feature_train, target_train)
test_predictions=model.predict(feature_test)
print('Тестовая выборка:', accuracy_score(target_test, test_predictions))

Тестовая выборка: 0.7107309486780715


### Мини вывод
- На тестовой выборке порог в 0.75 так же проходит случайный лес и дерево, а вот логистическая регрессия, увы, мимо.

# Вывод 

- Открыл файл
- Посмотрел файл на исправность 
- Разделил исходные данные на обущающую, валидационнуюи тестовую выборку 
- Лучший показатель по исследоваемым моделям оказалась дерево и случайный лес, логистическая регрессия показала низкий результат
- Хороший показатель на тестовой выборке показало модель дерева и случайный лес
- Используемые гиперпарамерты для случайного леса: `min_samples_leaf=2` `n_estimators=8`


# Чек-лист готовности проекта
- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75